In [8]:
import tensorflow as tf
import os
import sys
import json
import argparse
from PIL import Image

os.environ["CUDA_VISIBLE_DEVICES"] = "3"
sys.path.append("..")

from demo.image_similarity_keras.model import SiameseModel

In [9]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

model_path = "../demo/models/ConvNext_Large_64b_100ep_final"
augmentation_config = "../demo/configs/default_augmentation.json"

# Load model config
with open(os.path.join(model_path, "configs.json"), "r") as f:
    model_config = json.load(f)

    # Convert to Namespace
    model_config_ns = argparse.Namespace(**model_config)

# Load augmentation config
with open(augmentation_config, "r") as f:
    augmentation_config = json.load(f)

# Convert model_config dictionary to a namespace
model_config_ns = argparse.Namespace(**model_config)

# Get the image_size from model_config or use default value if missing
default_image_size = 224
image_size = model_config.get('image_size', default_image_size)

# Initialize model
model = SiameseModel(**model_config)

# Build and compile model
model.build(False)

# Load weights
model.model.load_weights(os.path.join(model_path, "weights"))

2023-11-01 07:52:45.066800: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-01 07:52:45.548980: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20651 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:5e:00.0, compute capability: 8.6


Model: "siamese_ConvNext_Large"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ConvNext_Large (KerasLayer)  (None, 1536)             196230336 
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 out_emb (Dense)             (None, 128)               32896     
                                                                 
 l2_norm (Lambda)            (None, 128)               0         
                                                                 
Total params: 197,181,504
Trainable params: 951,168
Non-trainable params: 196,230,336
_________________________________________________________________


In [10]:
def preprocess_image(image_path):
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        img = img.resize((224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    return img_array

In [11]:
def get_model_inference(img_array, model):
    return model.predict(tf.expand_dims(img_array, axis=0))

In [12]:
def calculate_similarity(feature1, feature2):
    similarity = tf.keras.losses.CosineSimilarity(axis=1)
    score = (similarity(feature1, feature2) + 1) / 2.0
    return score.numpy()

In [13]:
def compare_images(img_path1, img_path2, model):
    img1_array = preprocess_image(img_path1)
    img2_array = preprocess_image(img_path2)

    feature1 = get_model_inference(img1_array, model)
    feature2 = get_model_inference(img2_array, model)

    similarity_score = calculate_similarity(feature1, feature2)
    return similarity_score

In [14]:
img1_path = "path_to_image1.jpg"
img2_path = "path_to_image2.jpg"

similarity = compare_images(img1_path, img2_path, model)
print(f"Images similarity score: {similarity:.4f}")

2023-11-01 07:54:01.706205: I tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Loaded cuDNN version 8400
2023-11-01 07:54:03.107323: I tensorflow/stream_executor/cuda/cuda_blas.cc:1804] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Images similarity score: 0.3895
